In [ ]:
#add mbes_sim to search path
import sys
sys.path.insert(0, "../src/")

In [ ]:
#!/usr/bin/env python3
import os
import shutil as sh

#remove previously cached hdf5 files
for r,d,f in os.walk('./simresults/'):
  for file in f:
    if file.endswith('.h5'):
      of = r + "/" + file
      print(of)
      os.remove(of)


In [ ]:
%matplotlib widget

from matplotlib import pyplot as plt

import importlib
from copy import deepcopy
from collections import defaultdict
import numpy as np
import pandas as pd
import time
from tqdm.auto import tqdm

import view_functions as vf

In [ ]:
# simplify plotting
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 100
close_plots = True


def create_figure(name):
    if close_plots: plt.close(name)
    fig = plt.figure(name)
    fig.suptitle = name
    
    return fig

In [ ]:
!pwd

In [ ]:
importlib.reload(vf)
files = vf.find_files(
    './simresults/',
    search_for = [
        'prefix[sim3'
    ]
)

#print(len(files))

In [ ]:
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

vf.cache_files(files,hdf5=True)
    

In [ ]:
def resample_df(df, stepsize, sort_by):
           
    ret_df = pd.DataFrame()
    
    sort_df = df.sort_values(sort_by,ignore_index=True)
        
    samples = np.arange(min(sort_df[sort_by]),max(sort_df[sort_by]),stepsize)

    R = defaultdict(list)
    
    lens=[]
    for s in samples:
        data = sort_df[sort_df[sort_by] > s-stepsize/2]
        data = data[data[sort_by] <= s+stepsize/2]
        if len(data) > 0:
            lens.append(len(data))
            
    min_len = int(np.quantile(lens,0.1))
    for s in tqdm(samples):
        data = sort_df[sort_df[sort_by] > s-stepsize/2]
        data = data[data[sort_by] <= s+stepsize/2]
        if len(data) >= min_len:
            data = data.sample(n=min_len)
            
        if len(data) > 0:
            for k in data.keys():
                try:
                    d = data[k].mean()
                    R[k].append(d)
                except:
                    pass
    
    for k,v in R.items():
        ret_df[k] = v
        ret_df = ret_df.copy()
        
    return ret_df.copy()
    
#dat = resample_df(filtered_simreturns,2,'target_rx_angle')

In [ ]:
importlib.reload(vf)

def plot_simreturns(ax,label,df, sort_by=None, methods=None, methods_not=None,kwargs_list=[],xlim=None,ylim=None, hist_n=0,hist_kwargs_list={}):
    
    if methods is None:
        methods = df['methods'].to_numpy()[1].copy()

    if methods_not is not None:
        for m in methods_not:
            for i, meth in enumerate(methods):
                if meth == m:
                    del methods[i]
                    break

    if hist_n > 0:
        hist_fig = create_figure('hist-'+label)
        hist_ax = hist_fig.subplots()
        hist_ax.set_title('hist-'+label)

        for nr, method in enumerate(methods):
            kwargs = {}
            if len(hist_kwargs_list) > nr:
                kwargs = hist_kwargs_list[nr]
            hist_ax.hist(df[method],hist_n, label=method, **kwargs)

        hist_ax.set_xlabel('measured_sigma')
        hist_ax.legend()
                    
    if sort_by is None:
        for nr,method in enumerate(methods):
            kwargs={}
            if len(kwargs_list) > nr:
                kwargs = kwargs_list[nr]
            ax.plot(df.index, df[method], label=label + '-' + method,**kwargs)

        ax.set_xlabel('simulation nr.')

    else:
        df_copy = df.sort_values(sort_by)
        for nr,method in enumerate(methods):
            kwargs={}
            if len(kwargs_list) > nr:
                kwargs = kwargs_list[nr]
            ax.plot(df_copy[sort_by], df_copy[method], label=label + '-' + method,**kwargs)

        ax.set_xlabel(sort_by)
        
    if ylim is not None:
        ax.set_ylim(ylim)
    if xlim is not None:
        ax.set_xlim(xlim)
        
    ax.set_ylabel('measured 𝜎')
    ax.legend()
    
    ax.axhline(y=1,linestyle='--',c='red',linewidth=1.5)

    return ax

def plot_rolling(ax,label,df,roll_size,roll_win=None, sort_by=None, methods=None, methods_not=None,kwargs_list=[],xlim=None,ylim=None):
    
    if methods is None:
        methods = df['methods'].to_numpy()[1].copy()

    if methods_not is not None:
        for m in methods_not:
            for i, meth in enumerate(methods):
                if meth == m:
                    del methods[i]
                    break
                    
    if sort_by is None:
        rolling_df = df.rolling(roll_size, win_type=None).median()
        
        for nr,method in enumerate(methods):
            kwargs={}
            if len(kwargs_list) > nr:
                kwargs = kwargs_list[nr]
            ax.plot(rolling_df.index, rolling_df[method], label=label + '-' + method,**kwargs)

        ax.set_xlabel('simulation nr.')

    else:
        #df_copy = df.sort_values(sort_by)
        
        rolling_df = resample_df(df,roll_size,sort_by=sort_by)
        
        for nr,method in enumerate(methods):
            kwargs={}
            if len(kwargs_list) > nr:
                kwargs = kwargs_list[nr]
            ax.plot(rolling_df[sort_by], rolling_df[method], label=label + '-' + method,**kwargs)

        ax.set_xlabel(sort_by)
        
    if ylim is not None:
        ax.set_ylim(ylim)
    if xlim is not None:
        ax.set_xlim(xlim)

    ax.set_ylabel('measured 𝜎')
    ax.legend()
    
    ax.axhline(y=1,linestyle='--',c='red',linewidth=1.5)

    return ax

In [ ]:
beampatternlabels = []
voxellabels = []
#exaclabels = []
prefixlabels = []
surveypathlabels = []
bubblelabels = []

for file in files:
    if 'resfactor[1]' in file:
        if 'idealizedBeampattern[True]' in file: beampatternlabels.append('ideal')
        elif 'windowType[Exponential]' in file: beampatternlabels.append('exp')
        elif 'windowType[Box]' in file: beampatternlabels.append('box')
        elif 'windowType[Hann]' in file: beampatternlabels.append('hann')
    else:
        if 'windowType[Exponential]' in file: beampatternlabels.append('lowres')
        else:
            beampatternlabels.append('other')

    if 'bubbleType[SingleBubble]' in file:
        bubblelabels.append('singlebubble')
    else:
        bubblelabels.append('bubblestream')
    
    if 'FromNavigation' in file:
        surveypathlabels.append('realmotion-' + file.split('exagHPR[')[1].split(']')[0])
    else:
        surveypathlabels.append('idealmotion')
    
    voxellabels.append(file.split('voxelsize[')[1].split(']')[0])
    voxellabels[-1] = voxellabels[-1].replace('1.0','1')
    #exaclabels.append(file.split('exagHPR[')[1].split(']')[0])
    prefixlabels.append(file.split('prefix[')[1].split(']')[0])
    

sortarray = np.array(list(zip(voxellabels,beampatternlabels,surveypathlabels)))
ind = list(reversed(np.lexsort((sortarray[:,0],sortarray[:,1],sortarray[:,2]))))

files = np.array(files)[ind]
beampatternlabels = np.array(beampatternlabels)[ind]
#exaclabels = np.array(exaclabels)[ind]
voxellabels = np.array(voxellabels)[ind]
prefixlabels = np.array(prefixlabels)[ind]
surveypathlabels = np.array(surveypathlabels)[ind]

print(beampatternlabels)
print(surveypathlabels)
print(voxellabels)
print(prefixlabels)

In [ ]:
x = []
y = []
h = []
ya = []
pi = []
ro = []
for nr,(file,beampatternlabel,voxellabel,surveypathlabel,prefixlabel,bubblelabel) in enumerate(zip(files,beampatternlabels,voxellabels,surveypathlabels,prefixlabels, bubblelabels)):
    # if not beampatternlabel in ['exp']:
    #     continue
        
    # if not voxellabel in ['3']:
    #     continue
        
    # if not surveypathlabel in ['idealmotion',]:
    #     continue

    # if bubblelabel != 'singlebubble':
    #     continue

    setup, simreturns = vf.open_file(file,no_plots=True, verbose = False)
    for survey in simreturns['survey']:
        x.extend(survey.pingpositions_x)
        y.extend(survey.pingpositions_y)
        h.extend(survey.heaves)
        ya.extend(survey.yaws_degree)
        pi.extend(survey.pitchs_degree)
        ro.extend(survey.rolls_degree)

print(f'X:{np.min(x)} / {np.max(x)}')
print(f'Y:{np.min(y)} / {np.max(y)}')
print(f'Heaves:{np.min(h)} / {np.max(h)}')
print(f'yaws:{np.min(ya)} / {np.max(ya)}')
print(f'Pitchs:{np.min(pi)} / {np.max(pi)}')
print(f'Rolls:{np.min(ro)} / {np.max(ro)}')

In [ ]:
# plot 1 compare blockmean and weighted mean
importlib.reload(vf)

# set limited volume
local_min_range = 45
local_max_range = 120
local_min_rx_angle = -50
local_max_rx_angle = 50

local_n_samples=None
#local_n_samples=1000
#local_n_samples=800

fig = create_figure('compare block mean voxel block mean')
ax = fig.subplots()

#ax.axvline(40.93,linestyle='--',c='grey',linewidth=1.5)
#ax.axvline(27.12,linestyle='--',c='black',linewidth=1.5)
#ax.axvline(120,linestyle='--',c='lightgrey',linewidth=1.5)

methods = ['sv','sv_int_lin']

kwargs_list = [
    {'c':'grey', 'linewidth':'1.0'},
    {'c':'black', 'linewidth':'1.5'},
]

hist_kwargs_list = [
{'color':'grey'},
{'color':'black'},
]

#for nr,(file,beampatternlabel) in enumerate(zip(files,['box','ideal','exp','hann'])):
kwargs_nr = -1
for nr,(file,beampatternlabel,voxellabel,surveypathlabel,prefixlabel,bubblelabel) in enumerate(zip(files,beampatternlabels,voxellabels,surveypathlabels,prefixlabels, bubblelabels)):
    if not beampatternlabel in ['exp']:
        continue
        
    if not voxellabel in ['3']:
        continue
        
    if not surveypathlabel in ['idealmotion',]:
        continue

    if bubblelabel != 'singlebubble':
        continue
        
    labelname = '({}) {}/[{}m]/[{}]'.format(prefixlabel,beampatternlabel,voxellabel,surveypathlabel)
        
    kwargs_nr += 1
    if kwargs_nr >= len(kwargs_list): kwargs_nr=0
        
    
    setup, simreturns = vf.open_file(file,no_plots=True, verbose = False)
    filtered_simreturns = vf.filter_simreturns(simreturns,local_min_range,local_max_range,local_min_rx_angle,local_max_rx_angle).reset_index()
    #filtered_simreturns = deepcopy(simreturns)
                                               
    if local_n_samples is not None:
        filtered_simreturns = filtered_simreturns.sample(n=local_n_samples).reset_index()
    
    plot_simreturns(ax,label=labelname,df=filtered_simreturns,methods=methods,sort_by=None,
                    kwargs_list = kwargs_list,
                    ylim = [-0.1,2.1],
                     hist_n=0,
                    hist_kwargs_list=hist_kwargs_list
                    )
    
    
    vf.print_simreturns(filtered_simreturns, max_db_diffs=[], prefix=labelname+':\t',methods=methods,
                       show_db=False)     


In [ ]:
# plot 2 compare blockmean and weighted mean
importlib.reload(vf)

# set limited volume
local_min_range = 45
local_max_range = 120
local_min_rx_angle = -60
local_max_rx_angle = 60

local_n_samples=None
#local_n_samples=1000
#local_n_samples=800

fig = create_figure('View sorted by angle')
ax = fig.subplots()

#ax.axvline(40.93,linestyle='--',c='grey',linewidth=1.5)
#ax.axvline(27.12,linestyle='--',c='black',linewidth=1.5)
#ax.axvline(120,linestyle='--',c='lightgrey',linewidth=1.5)

methods = ['sv_int_lin']
sort_by = 'target_rx_angle'

kwargs_list = [
    {'c':'grey', 'linewidth':'1.0'},
    {'c':'black', 'linewidth':'1.5'},
]

hist_kwargs_list = [
{'color':'grey'},
{'color':'black'},
]

#for nr,(file,beampatternlabel) in enumerate(zip(files,['box','ideal','exp','hann'])):
kwargs_nr = -1
for nr,(file,beampatternlabel,voxellabel,surveypathlabel,prefixlabel,bubblelabel) in enumerate(zip(files,beampatternlabels,voxellabels,surveypathlabels,prefixlabels, bubblelabels)):
    if not beampatternlabel in ['box','hann']:
        continue
        
    if not voxellabel in ['3']:
        continue
        
    if not surveypathlabel in ['idealmotion',]:
        continue

    if bubblelabel != 'singlebubble':
        continue
        
    labelname = '({}) {}/[{}m]/[{}]'.format(prefixlabel,beampatternlabel,voxellabel,surveypathlabel)
        
    kwargs_nr += 1
    if kwargs_nr >= len(kwargs_list): kwargs_nr=0
        
    
    setup, simreturns = vf.open_file(file,no_plots=True, verbose = False)
    filtered_simreturns = vf.filter_simreturns(simreturns,local_min_range,local_max_range,local_min_rx_angle,local_max_rx_angle).reset_index()
    #filtered_simreturns = deepcopy(simreturns)
                                               
    if local_n_samples is not None:
        filtered_simreturns = filtered_simreturns.sample(n=local_n_samples).reset_index()

    if False:
        plot_simreturns(ax,label=labelname,df=filtered_simreturns,methods=methods,sort_by=sort_by,
                        kwargs_list = kwargs_list,
                        ylim = [-0.1,2.1],
                         hist_n=0,
                        hist_kwargs_list=hist_kwargs_list
                        )
    else:
        sym_filtered_simreturns = filtered_simreturns.copy()
        sym_filtered_simreturns['target_rx_angle'] = np.abs(filtered_simreturns['target_rx_angle'])
        plot_rolling(ax,label=labelname,df=sym_filtered_simreturns,methods=methods,sort_by='target_rx_angle',
                        kwargs_list = [kwargs_list[kwargs_nr]],
                        ylim = [0.98,1.04],
                     xlim = [0,65],
                     roll_size=2
                        )
    
    
    vf.print_simreturns(filtered_simreturns, max_db_diffs=[], prefix=labelname+':\t',methods=methods,
                       show_db=False)     


In [ ]:
# plot 3 compare blockmean and weighted mean
importlib.reload(vf)

# set limited volume
local_min_range = 0
local_max_range = 125
local_min_rx_angle = -50
local_max_rx_angle = 50

local_n_samples=None
#local_n_samples=1000
#local_n_samples=800

fig = create_figure('Sort by range')
ax = fig.subplots()

ax.axvline(40.93,linestyle='--',c='grey',linewidth=1.5)
ax.axvline(27.12,linestyle='--',c='black',linewidth=1.5)
ax.axvline(120,linestyle='--',c='lightgrey',linewidth=1.5)

methods = ['sv_int_lin']

sort_by = 'target_range'

kwargs_list = [
    {'c':'black', 'linewidth':'1.5'},
    {'c':'grey', 'linewidth':'1.0'},
]

hist_kwargs_list = [
{'color':'grey'},
{'color':'black'},
]

#for nr,(file,beampatternlabel) in enumerate(zip(files,['box','ideal','exp','hann'])):
kwargs_nr = -1
for nr,(file,beampatternlabel,voxellabel,surveypathlabel,prefixlabel,bubblelabel) in enumerate(zip(files,beampatternlabels,voxellabels,surveypathlabels,prefixlabels, bubblelabels)):
    if not beampatternlabel in ['box', 'hann']:
        continue
        
    if not voxellabel in ['3']:
        continue
        
    if not surveypathlabel in ['idealmotion',]:
        continue

    if bubblelabel != 'singlebubble':
        continue
        
    labelname = '({}) {}/[{}m]/[{}]'.format(prefixlabel,beampatternlabel,voxellabel,surveypathlabel)
        
    kwargs_nr += 1
    if kwargs_nr >= len(kwargs_list): kwargs_nr=0
        
    
    setup, simreturns = vf.open_file(file,no_plots=True, verbose = False)
    filtered_simreturns = vf.filter_simreturns(simreturns,local_min_range,local_max_range,local_min_rx_angle,local_max_rx_angle).reset_index()
    #filtered_simreturns = deepcopy(simreturns)
                                               
    if local_n_samples is not None:
        filtered_simreturns = filtered_simreturns.sample(n=local_n_samples).reset_index()
    
    plot_simreturns(ax,label=labelname,df=filtered_simreturns,methods=methods,sort_by=sort_by,
                    kwargs_list = [kwargs_list[kwargs_nr]],
                    ylim = [-0.1,2.1],
                    )
    
    
    vf.print_simreturns(filtered_simreturns, max_db_diffs=[], prefix=labelname+':\t',methods=methods,
                       show_db=False)     


In [ ]:
# plot 4 ctarget range real motion
importlib.reload(vf)

# set limited volume
local_min_range = 45
local_max_range = 120
local_min_rx_angle = -50
local_max_rx_angle = 50

local_n_samples=None
#local_n_samples=1000
#local_n_samples=800

fig = create_figure('Sort by range real motion 3m grid')
ax = fig.subplots()

# ax.axvline(40.93,linestyle='--',c='grey',linewidth=1.5)
# ax.axvline(27.12,linestyle='--',c='black',linewidth=1.5)
# ax.axvline(120,linestyle='--',c='lightgrey',linewidth=1.5)

methods = ['sv_int_lin']

sort_by = 'target_range'

kwargs_list = [
    {'c':'grey', 'linewidth':'1','zorder' : 1},
    {'c':'black', 'linewidth':'1.5','zorder' : 10},
]

hist_kwargs_list = [
{'color':'grey'},
{'color':'black'},
]

kwargs_nr = -1
for nr,(file,beampatternlabel,voxellabel,surveypathlabel,prefixlabel,bubblelabel) in enumerate(zip(files,beampatternlabels,voxellabels,surveypathlabels,prefixlabels, bubblelabels)):
    if not beampatternlabel in ['exp']:
        continue
        
    if not voxellabel in ['3']:
        continue
        
    if not surveypathlabel in ['realmotion-3','idealmotion']:
        continue

    if bubblelabel != 'singlebubble':
        continue
        
    labelname = '({}) {}/[{}m]/[{}]'.format(prefixlabel,beampatternlabel,voxellabel,surveypathlabel)
        
    kwargs_nr += 1
    if kwargs_nr >= len(kwargs_list): kwargs_nr=0
        
    
    setup, simreturns = vf.open_file(file,no_plots=True, verbose = False)
    filtered_simreturns = vf.filter_simreturns(simreturns,local_min_range,local_max_range,local_min_rx_angle,local_max_rx_angle).reset_index()
    #filtered_simreturns = deepcopy(simreturns)
                                               
    if local_n_samples is not None:
        filtered_simreturns = filtered_simreturns.sample(n=local_n_samples).reset_index()
    
    plot_simreturns(ax,label=labelname,df=filtered_simreturns,methods=methods,sort_by=sort_by,
                    kwargs_list = [kwargs_list[kwargs_nr]],
                    ylim = [0.8,1.4],
                    xlim = [44,121]
                    )
    
    
    vf.print_simreturns(filtered_simreturns, max_db_diffs=[], prefix=labelname+':\t',methods=methods,
                       show_db=False)     


In [ ]:
# plot 4 ctarget range real motion
importlib.reload(vf)

# set limited volume
local_min_range = 45
local_max_range = 120
local_min_rx_angle = -50
local_max_rx_angle = 50

local_n_samples=None
#local_n_samples=1000
#local_n_samples=800

fig = create_figure('Sort by range real motion 1m grid')
ax = fig.subplots()

# ax.axvline(40.93,linestyle='--',c='grey',linewidth=1.5)
# ax.axvline(27.12,linestyle='--',c='black',linewidth=1.5)
# ax.axvline(120,linestyle='--',c='lightgrey',linewidth=1.5)

methods = ['sv_int_lin']

sort_by = 'target_range'

kwargs_list = [
    {'c':'grey', 'linewidth':'1','zorder' : 1},
    {'c':'black', 'linewidth':'1.5','zorder' : 10},
]

hist_kwargs_list = [
{'color':'grey'},
{'color':'black'},
]

#for nr,(file,beampatternlabel) in enumerate(zip(files,['box','ideal','exp','hann'])):
kwargs_nr = -1
for nr,(file,beampatternlabel,voxellabel,surveypathlabel,prefixlabel,bubblelabel) in enumerate(zip(files,beampatternlabels,voxellabels,surveypathlabels,prefixlabels, bubblelabels)):
    if not beampatternlabel in ['exp']:
        continue
        
    if not voxellabel in ['1']:
        continue
        
    if not surveypathlabel in ['realmotion-3','idealmotion']:
        continue

    if bubblelabel != 'singlebubble':
        continue

    print(file)
        
    labelname = '({}) {}/[{}m]/[{}]'.format(prefixlabel,beampatternlabel,voxellabel,surveypathlabel)
        
    kwargs_nr += 1
    if kwargs_nr >= len(kwargs_list): kwargs_nr=0
        
    
    setup, simreturns = vf.open_file(file,no_plots=True, verbose = False)
    filtered_simreturns = vf.filter_simreturns(simreturns,local_min_range,local_max_range,local_min_rx_angle,local_max_rx_angle).reset_index()
    #filtered_simreturns = deepcopy(simreturns)
                                               
    if local_n_samples is not None:
        filtered_simreturns = filtered_simreturns.sample(n=local_n_samples).reset_index()
    
    plot_simreturns(ax,label=labelname,df=filtered_simreturns,methods=methods,sort_by=sort_by,
                    kwargs_list = [kwargs_list[kwargs_nr]],
                    ylim = [0.8,1.4],
                    xlim = [44,121]
                    )
    
    
    vf.print_simreturns(filtered_simreturns, max_db_diffs=[], prefix=labelname+':\t',methods=methods,
                       show_db=False)     


In [ ]:
# plot 45 bubble streams
importlib.reload(vf)

# set limited volume
local_min_range = 45
local_max_range = 120
local_min_rx_angle = -50
local_max_rx_angle = 50

layer_depths = [42, 60, 108]
layer_sizes  = [12, 12,  12]

local_n_samples=None
#local_n_samples=1000
#local_n_samples=800

#fig = create_figure('Sort by y distance')
#ax = fig.subplots()


methods = ['sv_int_lin']

sort_by = 'target_y'

kwargs_list = [
    {'c':'dimgray', 'linewidth':'1','zorder' : 10},
    {'c':'lightgray', 'linewidth':'1','zorder' : 1},
    {'c':'black', 'linewidth':'1','zorder' : 100},
]

hist_kwargs_list = [
{'color':'grey'},
{'color':'gray'},
{'color':'black'},
]

#for nr,(file,beampatternlabel) in enumerate(zip(files,['box','ideal','exp','hann'])):
kwargs_nr = -1
for nr,(file,beampatternlabel,voxellabel,surveypathlabel,prefixlabel,bubblelabel) in enumerate(zip(files,beampatternlabels,voxellabels,surveypathlabels,prefixlabels, bubblelabels)):
    if not beampatternlabel in ['exp']:
        continue
        
    if not voxellabel in ['1']:
        continue
        
    if not surveypathlabel in ['realmotion-1']:
        continue

    if bubblelabel != 'bubblestream':
        continue

    print(file)
        
    labelname = '({}) {}/[{}m]/[{}]'.format(prefixlabel,beampatternlabel,voxellabel,surveypathlabel)
        
    kwargs_nr += 1
    if kwargs_nr >= len(kwargs_list): kwargs_nr=0
        
    
    setup, simreturns = vf.open_file(file,no_plots=True, verbose = False)
    filtered_simreturns = vf.filter_simreturns(simreturns,local_min_range,local_max_range,local_min_rx_angle,local_max_rx_angle).reset_index()
    filtered_simreturns = deepcopy(simreturns)
                                               
    if local_n_samples is not None:
        filtered_simreturns = filtered_simreturns.sample(n=local_n_samples).reset_index()
    
    ax = vf.plot_simreturns_stream(create_figure,setup,name="Bubble stream by y distance",df=filtered_simreturns,methods=methods,sort_by=sort_by,
                    kwargs_list = kwargs_list,
                              layerDepths=layer_depths,
                              layerSizes=layer_sizes
                    #ylim = [0.8,1.4],
                    )

    ax.set_ylim(0.85,1.15)
    
    ax.axhline(1,linestyle='--',c='red',linewidth=1.5)
    ax.axvline(49,linestyle='--',c='black',linewidth=1.5)
    ax.axvline(-49,linestyle='--',c='black',linewidth=1.5)
    ax.axvline(62,linestyle='--',c='grey',linewidth=1.5)
    ax.axvline(-62,linestyle='--',c='grey',linewidth=1.5)
    ax.axvline(93,linestyle='--',c='lightgrey',linewidth=1.5)
    ax.axvline(-93,linestyle='--',c='lightgrey',linewidth=1.5)
    
    
    vf.print_simreturns(filtered_simreturns, max_db_diffs=[], prefix=labelname+':\t',methods=methods,
                       show_db=False)     

print("petger")